In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv').fillna(' ')
test_data = pd.read_csv('../input/test.csv').fillna(' ')
full_data = [train_data, test_data]

train_data.head(10)

In [ ]:
test_data.head(10)

* Get all words of comments

In [ ]:
import re
words = set()
word_re = re.compile('\w+\'*\w*')

> words_list = list(words)
> print(words_list[:100])

* Get frequencies of each word, and replace uncommon word with 0

words_count = dict()
word_re = re.compile('\w+\'*\w*')
for data_set in full_data:
    for comment in data_set['comment_text']:
        comment_words = re.findall(word_re, comment)
        if not comment_words is None:
            for word in comment_words:
                word = word.lower()
                if not word in words_count:
                    words_count[word] = 0
                words_count[word] += 1
print(len(words_count))

sorted_words = sorted(words_count.items(), key = lambda x: x[1], reverse=True)
print(sorted_words[:100])

word_df = pd.DataFrame()
word_text = []
word_count = []
for word_tuple in sorted_words:
    word_text.append(word_tuple[0])
    word_count.append(word_tuple[1])
word_df['word'] = word_text
word_df['count'] = word_count
word_df.head(10)

print(word_df['count'].mean())
print(word_df['count'].std())
print(word_df['count'].max())

word_df['count'][100000:].hist()

In [ ]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
word_infos = {}
for row in train_data.as_matrix():
    cid = row[0]
    comment = row[1]
    cwords = re.findall(word_re, comment)
    if not cwords is None:
        for word in cwords:
            word = word.lower()
            if not word in word_infos:
                word_infos[word] = [0, 0, 0, 0, 0, 0, 0]
            word_infos[word][0] += 1
            for i in range(6):
                word_infos[word][i+1] += row[i+2]
print(len(word_infos))

In [ ]:
cclass_infos = {}
cclass_infos['word'] = []
cclass_infos['count'] = []
for cclass in classes:
    cclass_infos[cclass] = []
for word in word_infos:
    cclass_infos['word'].append(word)
    cclass_infos['count'].append(word_infos[word][0])
    i = 1
    for cclass in classes:
        cclass_infos[cclass].append(word_infos[word][i])
        i += 1
word_info_df = pd.DataFrame(cclass_infos)

In [ ]:
word_info_df.head(10)

In [ ]:
for cclass in classes:
    word_info_df[cclass+'_rate'] = word_info_df[cclass] / word_info_df['count']

In [ ]:
word_info_df.head(20)

word_info_df.sort_values(by=['toxic_rate', 'count'], ascending=False)

word_info_df.set_index('word')

In [ ]:
word_rate_info = word_info_df.set_index('word').T.to_dict('dict')

print(word_rate_info['the'])

In [ ]:
for cclass in classes:
    test_data[cclass] = 0

In [ ]:
test_data.head(10)

test_data.shape

In [ ]:
limited_words_info = {}
for word in word_rate_info:
    if word_rate_info[word]['count'] < 3 :
        continue
    max_rate = 0
    for cclass in classes:
        max_rate = max(max_rate, word_rate_info[word][cclass+'_rate'])
    if max_rate < 0.1:
        continue
    limited_words_info[word] = word_rate_info[word]
word_rate_info = limited_words_info
print(len(word_rate_info))

print(test_data['comment_text'][0])
comment = test_data['comment_text'][0]
cwords = re.findall(word_re, comment)
print(cwords)

print(test_data.shape[0])

In [ ]:
submission = {}
submission['id'] = []
for cclass in classes:
    submission[cclass] = []
for i in range(test_data.shape[0]):
    id_text = test_data['id'][i]
    comment = test_data['comment_text'][i]
    cwords = re.findall(word_re, comment)
    max_rate = {}
    for cclass in classes:
        max_rate[cclass] = 0
    if not cwords is None:
        for word in cwords:
            if word in word_rate_info:
                for cclass in classes:
                    max_rate[cclass] = max(max_rate[cclass], word_rate_info[word][cclass+'_rate'])
    submission['id'].append(id_text)
    for cclass in classes:
        submission[cclass].append(max_rate[cclass])

In [ ]:
test_data.head()

In [ ]:
submission_df = pd.DataFrame(submission)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.head()